In [1]:
import os
import warnings

warnings.filterwarnings(action="ignore")

from openff.toolkit.typing.engines.smirnoff import ForceField
from pkg_resources import resource_filename
from tqdm.auto import tqdm

from paprika.build.system.taproom import BuildTaproomAPR

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html
LICENSE: No product keys!
Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.


In [2]:
os.environ["INTERCHANGE_EXPERIMENTAL"] = "1"

* The `BuildTaproomAPR` class requires the OpenFF-Interchange modules. Install in your conda environment with:

`conda install -c conda-forge openff-interchange`

## 01) Build System in explicit solvent

In [3]:
# Define force field (OpenFF)
force_field = ForceField("openff-2.0.0.offxml")

In [4]:
system = BuildTaproomAPR(
    host_code="bcd",
    guest_code="hex",
    n_water=3000,
    force_field=force_field,
    working_folder="explicit_solvent",
    disable_progress=False,
)

In [5]:
# Build APR files -- parallelize over 4 CPUs
system.build_system(n_cpus=4, clean_files=True)

  0%|          | 0/46 [00:00<?, ?it/s]

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.
<frozen importlib._bootstrap>:228: RuntimeWarning: scipy.

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

## 02) Build System in implicit solvent

In [6]:
# Define force field (OpenFF)
gbsa = resource_filename(
    "openff.toolkit",
    os.path.join("data", "test_forcefields", "GBSA_OBC2-1.0.offxml"),
)
force_field = ForceField("openff-2.0.0.offxml", gbsa)

In [7]:
system = BuildTaproomAPR(
    host_code="acd",
    guest_code="bam",
    n_water=None, #<-- Set this to None to build system without water molecules
    force_field=force_field,
    working_folder="implicit_solvent",
    disable_progress=False,
)

In [8]:
# Build APR files -- parallelize over 4 CPUs
system.build_system(n_cpus=4, clean_files=True)

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

## 03) Generate files for multiple host-guest complexes

In [9]:
host_guest_codes = {
    "acd": ["bam", "cbu"],
    "bcd": ["ben", "hex"],
}

In [10]:
for host in tqdm(host_guest_codes, desc="Host"):
    for guest in tqdm(host_guest_codes[host], desc="Guests", leave=False):
        system = BuildTaproomAPR(
            host_code=host,
            guest_code=guest,
            n_water=None,
            force_field=force_field,
            build_folder=f"{host}-{guest}/build_files",
            working_folder=f"{host}-{guest}/simulations",
            disable_progress=True,
        )
        system.build_system(n_cpus=4)

Host:   0%|          | 0/2 [00:00<?, ?it/s]

Guests:   0%|          | 0/2 [00:00<?, ?it/s]

Guests:   0%|          | 0/2 [00:00<?, ?it/s]